In [1]:
# Generic: A Modular Multi-Pipeline Framework for Probability Fusion Ensembles
# Specific: Cross-Lingual Sentiment Analysis with Probability Fusion Ensembles: A Modular Multi-Pipeline Framework for Low-Resource Languages

import os
import pandas as pd
import pprint

from pathlib import Path
from transformers import pipeline

from src import sentiment, helper, utility, context
from src.config import *
from src.metrics import evaluate_pipe

os.environ["MAMBA_USE_MAMBAPY"] = MambaBackend.FORCE_CUDA
os.environ["TOKENIZERS_PARALLELISM"] = "false"
helper.list_config()

if AppConfig.INFER:
    sample_texts = [
        "Maganda ang serbisyo at mabilis ang delivery!",
        "Sobrang pangit ng karanasan ko.",
        "It was okay, nothing special.",
    ]
    sentiment.infer(sample_texts, MambaConfig)
    sentiment.infer(sample_texts, MBertConfig)
else:
    mbert_context = context.setup_pipeline(MBertConfig, require_translation = False)
    mbert_trainer = sentiment.train(mbert_context)
    
    mamba_context = context.setup_pipeline(MambaConfig, require_translation = True)
    mamba_trainer = sentiment.train(mamba_context)
    
    # TBA: FUSION #





### SAMPLE PREDICTIONS ###
[('Maganda ang serbisyo at mabilis ang delivery!', 1), ('Sobrang pangit ng karanasan ko.', 1), ('It was okay, nothing special.', 2)]

### SAMPLE PREDICTIONS ###
[('Maganda ang serbisyo at mabilis ang delivery!', 0), ('Sobrang pangit ng karanasan ko.', 1), ('It was okay, nothing special.', 2)]
